In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
config = PeftConfig.from_pretrained("NursNurs/outputs_gemma2b_yuso")
base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")
model = PeftModel.from_pretrained(base_model, "NursNurs/outputs_gemma2b_yuso")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
base_model.to('cuda:0')

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256, bias=False)
            (lora_dropout): ModuleDict(
    

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")


In [6]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
     -------------- ------------------------- 2.0/5.8 MB 44.1 MB/s eta 0:00:01
     ----------------------------------- ---- 5.1/5.8 MB 54.6 MB/s eta 0:00:01
     ---------------------------------------- 5.8/5.8 MB 53.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------------------- 4.1/4.1 MB 132.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 97.2 MB/s eta 0:00:26
     ---------------------------------------- 0.0/2.4 GB 108.8 MB/s eta 0:00:23
     ---------------------------------------- 0.0/2.4 GB 131.2 MB/s eta 0:00:19
     ---------------------------------------- 0.0/2.4 GB 108.8 MB/s eta 0:00:23
     ---------------------------------------- 0.0/2.4 GB 131.2 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Nursulu_1\\anaconda3\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [10]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"
  prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.\\n\\n
  You are given a topic. Your task is to generate a meme caption based on the topic. Only output the meme caption and nothing more.
  {query}
  <end_of_turn>\\n<start_of_turn>model
  
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=30, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [11]:
result = get_completion(query="Topic: a doctor", model=model, tokenizer=tokenizer)
print(result)


  Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n
  You are given a topic. Your task is to generate a meme caption based on the topic. Only output the meme caption and nothing more.
  Topic: a doctor
  \nmodel
  
  <p>doctor <sep> y u no see my leg!! <sep> Y R U SO SLOW!?  parlano parlano;  parlano parlano


In [8]:
input_text = "Topic: a doctor"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

C:\Users\Nursulu_1\AppData\Roaming\Python\Python311\site-packages\transformers\generation\utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
C:\Users\Nursulu_1\AppData\Roaming\Python\Python311\site-packages\transformers\models\gemma\modeling_gemma.py:482: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<bos>Topic: a doctor's appointment

You are given a topic. Your task is to write
